## AI Inference on image dataset retrieved from a cloud
This notebook is an example how to use DeGirum PySDK to do AI inference of an image dataset.
Image dataset is retrieved from the cloud using `fiftyone` API.

#### This sample uses the following external packages, which need to be installed:
1. **fiftyone**: `pip install fiftyone`

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](env.ini) file, located in the same directory as this notebook.

#### Specify dataset parameters here

In [ ]:
# specify the name of desired dataset to retrieve;
# see https://voxel51.com/docs/fiftyone/user_guide/dataset_zoo/datasets.html
foz_dataset_name = "coco-2017"

# specify list of class labels to retrieve; None for all classes
foz_classes = None # ["car", "cup", "person"]

# specify which splits to download ("train", "validation", "test")
foz_splits = "validation"

# specify # of samples in dataset to retrieve
samples_num = 1000

# specify the model to be used for inference
model_name = "yolo_v5s_coco--512x512_quant_n2x_orca1_1"

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, mytools

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

target = dg.CLOUD # <-- on the Cloud Platform
# target = mytools.get_ai_server_hostname() # <-- on AI Server deployed in your LAN
# target = dg.LOCAL # <-- on ORCA accelerator installed on this computer

# connect to AI inference engine getting zoo URL and token from env.ini file
zoo = dg.connect(target, mytools.get_cloud_zoo_url(), mytools.get_token())

#### The rest of the cells below should run without any modifications

In [ ]:
import sys, IPython
fo = mytools.import_fiftyone() # import 'fiftyone' package for dataset management

In [ ]:
# download dataset
dataset = fo.zoo.load_zoo_dataset(
    foz_dataset_name,
    dataset_dir="./my-dataset", 
    classes=foz_classes,
    split=foz_splits,
    max_samples=samples_num,
    shuffle=True,
    drop_existing_dataset=False)
print(dataset)

In [ ]:
# load model
model = zoo.load_model(model_name)
model.output_confidence_threshold = 0.1 # set low confidence threshold for proper statistics calculation

In [ ]:
print("Extracting dataset samples...")
all_samples = dataset.head(dataset.count()) # retrieve all dataset samples
all_files = [s.filepath for s in all_samples]
print("...done")

In [ ]:
print("Running inference:")
progress = mytools.Progress(len(all_files))
for n, res in enumerate(model.predict_batch(all_files)):
    if model.image_backend == 'pil':
        w, h = res.image.size
    else: # opencv
        w = res.image.shape[1]
        h = res.image.shape[0]
    
    detections = []
    for box in res.results:
        # Convert to [top-left-x, top-left-y, width, height]
        # in relative coordinates in [0, 1] x [0, 1]
        x1, y1, x2, y2 = box["bbox"]
        rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]
        detections.append(fo.Detection(label=box["label"], bounding_box=rel_box, confidence=box["score"]))
    all_samples[n]["predictions"] = fo.Detections(detections=detections)
    all_samples[n].save()
    progress.step()

In [ ]:
# run evaluation on predictions
eval_result = dataset.evaluate_detections("predictions", classes=foz_classes, compute_mAP=True)

In [ ]:
# print some evaluation results
print( f"mAP = {eval_result.mAP():.2f}\n")
eval_result.print_report(classes=foz_classes)